import time
plc=1
time.sleep(1500*plc)         #run after 1 houre = 3600 seconds

In [1]:
import os
import glob
import scipy
import random
import bisect
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import signal
from scipy.signal import find_peaks
import matplotlib.pyplot as plt
from preprocess import *                         #ماژول استخراج پنجره ها
from data import *                          #ماژول محلی ورود داده ها
from augment import *                            #ماژول های داده افزایی
from lstm_cnn import *                           #ماژول های داده افزایی با lstm_cnn
import augment
import importlib
importlib.reload(augment)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import BatchNormalization
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report,recall_score,precision_score
from tensorflow.keras import callbacks
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D,Conv1D,Dropout,MaxPooling1D,MaxPooling2D,Flatten,Dense
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
cwd = os.getcwd() #
fullscrn()

In [2]:
def maxindx(ar):
    ar_cpy=np.array(ar)
    pred=np.argmax(ar,axis=1)
    for i in range (len(pred)):
        if pred[i]==0:
            ar_cpy[i,0]=-1000
    pred=np.argmax(ar_cpy,axis=1)
    return(pred)

<h1><center><div style="direction:rtl;font-family:B Nazanin">Importing Data</div></center></h1>

In [3]:
cls_num=42

for i in range (1,cls_num+1):
    vars()['ecgH'+str(i)],vars()['ecg_tstH'+str(i)]=NonInvasiveFetalECGThorax1(i)
os.chdir(cwd)
clses_lens=np.array([])
clses_lensH=np.array([])
i=0                               #جمع آوری داده ها و چاپ تعداد نمونه ی هر کلاس
ecgH=np.array(ecgH1)
print(1,len(vars()['ecgH'+str(1)]), end='\t')
clses_lensH=np.append(clses_lensH,len(vars()['ecgH'+str(1)]))
for i in range (2,cls_num+1):
    ecgH=np.concatenate((ecgH,vars()['ecgH'+str(i)]),axis=0)
    clses_lensH=np.append(clses_lensH,len(vars()['ecgH'+str(i)]))
    print( i,len(vars()['ecgH'+str(i)]), end='\t')

mx_sigH=max(clses_lensH)
btch=int(max(np.sum(clses_lensH)/5,20))
print('\nshape ecgH=',np.shape(ecgH))

C:\ProgramData\Anaconda3\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


first_train_shape= (1800, 751)
first_test_shape= (1965, 751)
classes_quantity= 42
tr_lbls=	 {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42}
Count_labels= [36 52 45 40 48 44 36 50 46 56 36 39 46 42 40 39 52 45 40 49 43 40 35 41
 41 37 47 40 47 40 43 41 37 52 45 40 37 41 43 45 39 45]
max(train_feature_Altitude)= 42.0
min(train_feature_Altitude)= -5.7322181
first_train_sample=
 [ 3.3000000e+01  3.0577079e+00  2.9885631e+00  2.8214720e+00
  2.6392369e+00  2.4456015e+00  2.1598024e+00  1.7604417e+00
  1.2885524e+00  7.7969192e-01  2.5702086e-01 -2.4136675e-01
 -6.9723052e-01 -1.1682231e+00 -1.7182241e+00 -2.3618027e+00
 -3.0698550e+00 -3.7349237e+00 -4.2844303e+00 -4.7071089e+00
 -5.0354394e+00 -5.2173834e+00 -5.2658047e+00 -5.2337997e+00
 -5.1379266e+00 -5.0083663e+00 -4.8910492e+00 -4.7745101e+00
 -4.5631708e+00 -4.2865916e+00 -4.0250031e+00 -3.7163384e+00
 -3.3250977e+00 -2.957

1 36	2 52	3 45	4 40	5 48	6 44	7 36	8 50	9 46	10 56	11 36	12 39	13 46	14 42	15 40	16 39	17 52	18 45	19 40	20 49	21 43	22 40	23 35	24 41	25 41	26 37	27 47	28 40	29 47	30 40	31 43	32 41	33 37	34 52	35 45	36 40	37 37	38 41	39 43	40 45	41 39	42 45	
shape ecgH= (1800, 481)


In [4]:
# Vertical
for i in range (1,cls_num+1):
    vars()['ecgV'+str(i)],vars()['ecg_tstV'+str(i)]=NonInvasiveFetalECGThorax2(i)
os.chdir(cwd)
clses_lensV=np.array([])
i=0                               #جمع آوری داده ها و چاپ تعداد نمونه ی هر کلاس
ecgV=np.array(ecgV1)
print(1,len(vars()['ecgV'+str(1)]), end='\t')
clses_lens=np.append(clses_lens,len(vars()['ecgV'+str(1)]))
for i in range (2,cls_num+1):
    ecgV=np.concatenate((ecgV,vars()['ecgV'+str(i)]),axis=0)
    clses_lensV=np.append(clses_lensV,len(vars()['ecgV'+str(i)]))
    print( i,len(vars()['ecgV'+str(i)]), end='\t')

mx_sigV=max(clses_lensV)
print('\nshape ecgV=',np.shape(ecgV))

first_train_shape= (1800, 751)
first_test_shape= (1965, 751)
classes_quantity= 42
tr_lbls=	 {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42}
Count_labels= [36 52 45 40 48 44 36 50 46 56 36 39 46 42 40 39 52 45 40 49 43 40 35 41
 41 37 47 40 47 40 43 41 37 52 45 40 37 41 43 45 39 45]
max(train_feature_Altitude)= 42.0
min(train_feature_Altitude)= -5.4156817
first_train_sample=
 [ 3.3000000e+01  4.2513762e+00  4.2896489e+00  4.2280762e+00
  4.1292816e+00  3.9943007e+00  3.7654856e+00  3.4269719e+00
  3.0086621e+00  2.5496071e+00  2.0923628e+00  1.6631705e+00
  1.2298972e+00  7.4750542e-01  1.9019816e-01 -4.5405910e-01
 -1.1292452e+00 -1.7296598e+00 -2.2396899e+00 -2.7059539e+00
 -3.1589365e+00 -3.5353216e+00 -3.7954363e+00 -3.9314129e+00
 -3.9489070e+00 -3.9042555e+00 -3.8611114e+00 -3.8220483e+00
 -3.7560545e+00 -3.6858879e+00 -3.5917502e+00 -3.4210764e+00
 -3.1618318e+00 -2.893

1 36	2 52	3 45	4 40	5 48	6 44	7 36	8 50	9 46	10 56	11 36	12 39	13 46	14 42	15 40	16 39	17 52	18 45	19 40	20 49	21 43	22 40	23 35	24 41	25 41	26 37	27 47	28 40	29 47	30 40	31 43	32 41	33 37	34 52	35 45	36 40	37 37	38 41	39 43	40 45	41 39	42 45	
shape ecgV= (1800, 481)


In [5]:
for i in range (1,cls_num+1):
    #vars()['ecgHV'+str(i)]=np.array([])
    vars()['ecg'+str(i)]=np.array(np.concatenate((vars()['ecgH'+str(i)][:,0:-1],vars()['ecgV'+str(i)]),axis=1))
    vars()['ecg_tst'+str(i)]=np.array(np.concatenate((vars()['ecg_tstH'+str(i)][:,0:-1],vars()['ecg_tstV'+str(i)]),axis=1))
ecg=np.array(np.concatenate((ecgH[:,:-1],ecgV),axis=1))

In [6]:
np.shape(ecg)

(1800, 961)

<h2><center><div style="direction:rtl;font-family:B Nazanin">Base Train windows</div></center></h1>

In [7]:
smpl_rte=960                                              # در ماژول ها نیز همین مقدار ثبت شده
i=0
windws=np.array([])

for cls in range (1,cls_num+1):                                 #ساخت پنجره های داده های آموزش اصلی
    vars()['wndws'+str(cls)]=np.array(vars()['ecg'+str(cls)])  
    
print("each class and its windows =")

cls_wndws=np.array([])
for cls in range (1,cls_num+1):                                # آرایه ی تعداد پنجره ی هر کلاس
    wns=len(vars()['wndws'+str(cls)])
    cls_wndws=np.append(cls_wndws,wns)
    print(cls, wns, end='\t')

each class and its windows =
1 36	2 52	3 45	4 40	5 48	6 44	7 36	8 50	9 46	10 56	11 36	12 39	13 46	14 42	15 40	16 39	17 52	18 45	19 40	20 49	21 43	22 40	23 35	24 41	25 41	26 37	27 47	28 40	29 47	30 40	31 43	32 41	33 37	34 52	35 45	36 40	37 37	38 41	39 43	40 45	41 39	42 45	

<h1><center><div style="direction:rtl;font-family:B Nazanin">Data Augmentation</div></center></h1>

In [8]:
for alpha in range(1,7,2):
    if alpha==5:
        alpha=10
    print(alpha)


1
3
10


In [9]:
mx_wndws=int(np.max(cls_wndws)+1)   # حداکثر تعداد پنجره ی موجود بین کلاس ها
print('\n max =', mx_wndws)
rpt_cnn_clsfctn=3                 #تعداد اجرای شبکه عصبی برای میانگین گیری هر حالت 
aug_amnt_Ttl=np.empty((0,5), int) 
rslts=np.array([])
for X in range(1,6):                               #داده افزایی تا چند برابر کلاس اکثریت
    mx_wndws=int(np.max(cls_wndws))   # حداکثر تعداد پنجره ی موجود بین کلاس ها
    mx_wndws=int(X*mx_wndws)          #
    for alpha in range(1,4):
        rprt=np.array([])
        for beta in range(1,4):                        #چند برابر کردن داده افزایی هر کلاس متناسب ضعیف بودن آن کلاس
            if beta==3:
                beta=beta*2

            f_scr=np.array([86,93,92,92,93,92])                      #UnAg,Scl.2,Mag.05,Tm.2,Gs1,frqnc2,GAN,LSTMexpgdo
            f_scr_difrnc=f_scr-f_scr[0]
            for z in range (len(f_scr_difrnc)):
                if f_scr_difrnc[z]<0:
                    f_scr_difrnc[z]=0
            f_scr_difrnc=f_scr_difrnc**beta
            f_scr_rtio=f_scr_difrnc/(sum(f_scr_difrnc))
            aug_amnt=np.empty((0,len(f_scr_rtio)-1), int) 

            scors0=np.array([77,100,71,82,88,94,100,93,93,100,100,92,96,87,91,93,79,100,100,100,96,98,81,89,100,100,88,91,80,33,100,84,96,100,94,80,100,100,89,100,81,100])         #Class Recalls without Augmentation
            MxMn20fscr=((max(f_scr)-f_scr[0])/20)**0.7
            AgScr=1+alpha*MxMn20fscr*((100-scors0)/(np.max(100-scors0)))**beta        #Class Specific
            
            for i in range (1,cls_num+1):
                aug_amnt=np.append(aug_amnt,[f_scr_rtio[1:]*(mx_wndws*AgScr[i-1]-cls_wndws[i-1])],axis=0)
                #vlum_win=mx_wndws-cls_wndws[i-1]                        #میزان داده افزایی در روش های عمیق (پنجره)
            aug_amnt=np.int16(np.round(aug_amnt))
            aug_amnt_Ttl=np.append(aug_amnt_Ttl,aug_amnt,axis=0)
            #print(np.int16(100*f_scr_rtio))
            #print(aug_amnt)
            rprt=np.empty((0,3),float)
            print(aug_amnt)
            for repeat in range(1,rpt_cnn_clsfctn+1):
                xtrain=np.empty((0,smpl_rte+1), float) 
                for i in range (1,cls_num+1):                            #تعریف آرایه ی پنجره های کلاس ها
                    xtrain=np.append(xtrain,vars()['wndws'+str(i)],axis=0)
                    if aug_amnt[i-1,0]>0:
                        methd=10
                        sort=-3  #-5=Dis_sam_Cls=qlty1 #-4=var_Dis=qlty2 #-3=DisSm/DisOthr=qlty3  #-2=invrs_GDO_papr=qlty4   #-1=GDO_papr=qlty5
                        dp_slct_dstrb=4            #1-sequencial 2-linear 3-beta 4-exponential 5-unfrm
                        Hrzntl=srtd_dataH(methd,i,aug_amnt[i-1,0],dp_slct_dstrb,sort)
                        Vrtcl=srtd_dataV(methd,i,aug_amnt[i-1,0],dp_slct_dstrb,sort)
                        HrzntlVrtcl=np.concatenate((Hrzntl[:,:-1],Vrtcl),axis=1)
                        xtrain=np.append(xtrain,HrzntlVrtcl,axis=0)
                    if aug_amnt[i-1,1]>0:
                        methd=20
                        sort=-3
                        dp_slct_dstrb=2
                        Hrzntl=srtd_dataH(methd,i,aug_amnt[i-1,1],dp_slct_dstrb,sort)
                        Vrtcl=srtd_dataV(methd,i,aug_amnt[i-1,1],dp_slct_dstrb,sort)
                        HrzntlVrtcl=np.concatenate((Hrzntl[:,:-1],Vrtcl),axis=1)
                        xtrain=np.append(xtrain,HrzntlVrtcl,axis=0)
                    if aug_amnt[i-1,2]>0:
                        methd=30
                        sort=-3
                        dp_slct_dstrb=4
                        Hrzntl=srtd_dataH(methd,i,aug_amnt[i-1,2],dp_slct_dstrb,sort)
                        Vrtcl=srtd_dataV(methd,i,aug_amnt[i-1,2],dp_slct_dstrb,sort)
                        HrzntlVrtcl=np.concatenate((Hrzntl[:,:-1],Vrtcl),axis=1)
                        xtrain=np.append(xtrain,HrzntlVrtcl,axis=0)
                    if aug_amnt[i-1,3]>0:
                        methd=40
                        sort=-1
                        dp_slct_dstrb=4
                        Hrzntl=srtd_dataH(methd,i,aug_amnt[i-1,3],dp_slct_dstrb,sort)
                        Vrtcl=srtd_dataV(methd,i,aug_amnt[i-1,3],dp_slct_dstrb,sort)
                        HrzntlVrtcl=np.concatenate((Hrzntl[:,:-1],Vrtcl),axis=1)
                        xtrain=np.append(xtrain,HrzntlVrtcl,axis=0)
                    if aug_amnt[i-1,4]>0:
                        methd=50
                        sort=-3
                        dp_slct_dstrb=4
                        Hrzntl=srtd_dataH(methd,i,aug_amnt[i-1,4],dp_slct_dstrb,sort)
                        Vrtcl=srtd_dataV(methd,i,aug_amnt[i-1,4],dp_slct_dstrb,sort)
                        HrzntlVrtcl=np.concatenate((Hrzntl[:,:-1],Vrtcl),axis=1)
                        xtrain=np.append(xtrain,HrzntlVrtcl,axis=0)
                    '''if aug_amnt[i-1,5]>0:
                        methd=60
                        sort=-1
                        dp_slct_dstrb=4
                        Hrzntl=srtd_dataH(methd,i,aug_amnt[i-1,5],dp_slct_dstrb,sort)
                        Vrtcl=srtd_dataV(methd,i,aug_amnt[i-1,5],dp_slct_dstrb,sort)
                        HrzntlVrtcl=np.concatenate((Hrzntl[:,:-1],Vrtcl),axis=1)
                        xtrain=np.append(xtrain,HrzntlVrtcl,axis=0)
                    if aug_amnt[i-1,6]>0:
                        methd=70
                        sort=-1
                        dp_slct_dstrb=2
                        Hrzntl=srtd_dataH(methd,i,aug_amnt[i-1,6],dp_slct_dstrb,sort)
                        Vrtcl=srtd_dataV(methd,i,aug_amnt[i-1,6],dp_slct_dstrb,sort)
                        HrzntlVrtcl=np.concatenate((Hrzntl[:,:-1],Vrtcl),axis=1)
                        xtrain=np.append(xtrain,HrzntlVrtcl,axis=0)
                    #print('class ', i, 'train shape = ', np.shape(xtrain))'''

                # Normalization train windows
                mx_aug=np.max(xtrain[:,:-1])
                mx=np.max(ecg[:,:-1])
                mn_aug=np.min(xtrain[:,:-1])
                mn=np.min(ecg[:,:-1])
                xtrain[:,:-1]= 2*(xtrain[:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1


                ###Test Windows:
                wndws_test=np.empty((0,len(wndws1[0])), float)
                i=0
                windws=np.array([])
                for cls in range (1,cls_num+1):                                 #ساخت پنجره های داده های آموزش اصلی
                    vars()['wndws_tst'+str(cls)]=np.empty((0,len(wndws1[0])), float)
                    vars()['wndws_tst'+str(cls)]=np.append(vars()['wndws_tst'+str(cls)],vars()['ecg_tst'+str(cls)],axis=0)   

                # Normalization test windows
                for i in range (1,cls_num+1):                
                    cls=i
                    #print('\n cls', i, ' >> ')
                    #print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
                    #print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))
                    vars()['wndws_tst'+str(i)][:,:-1]= 2*(vars()['wndws_tst'+str(i)][:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1
                    #print('after normalizing >>')
                    #print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
                    #print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))
                #print("each class and its windows =")                    #تجمیع کلاس های تست
                for cls in range (1,cls_num+1):                                # آرایه ی تعداد پنجره ی هر کلاس
                    wns=len(vars()['wndws_tst'+str(cls)])
                    wndws_test=np.append(wndws_test,vars()['wndws_tst'+str(cls)],axis=0)
                Xtest=wndws_test[:,:-1]
                ytest=np.int16(wndws_test[:,-1])

                trainx=np.random.permutation(xtrain)
                Xtrain=np.array(trainx[:,:-1])
                ytrain=np.int16(trainx[:,-1])

                X_train=np.array(Xtrain)
                y_train=np.array(ytrain)
                X_test=np.array(Xtest)
                y_test=np.array(ytest)
                X_valid=np.array(Xtrain)
                y_valid=np.array(ytrain)
                #print('X_train => ', X_train.shape)
                #print('y_train => ', y_train.shape)
                #print('X_test  => ', X_test.shape)
                #print('y_test  => ', y_test.shape)
                #print('X_valid  => ', X_valid.shape)
                #print('y_valid  => ', y_valid.shape)

                y_train_cat=to_categorical(y_train)
                y_valid_cat=to_categorical(y_valid)
                y_test_cat=to_categorical(y_test)
                X_train = np.expand_dims(X_train, axis=2)
                X_valid = np.expand_dims(X_valid, axis=2)
                X_test = np.expand_dims(X_test, axis=2)

                ####### NETWORK #######
                accuracy=0
                acc_crs=np.array([])
                ##for i in range (10):
                ##    if accuracy<0.2 :
                verbose, epochs, batch_size = 0, 1000, btch
                n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train_cat.shape[1]
                steps_per_epoch = len(X_train)//batch_size
                validation_steps = len(X_valid)//batch_size # if you have test data
                model_crs = Sequential()
                BatchNormalization()
                model_crs.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features)))
                #model_crs.add(MaxPooling1D(pool_size=3))
                BatchNormalization()
                model_crs.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
                #model_crs.add(Dropout(0.5))
                model_crs.add(MaxPooling1D(pool_size=3))
                BatchNormalization()
                model_crs.add(Conv1D(filters=256, kernel_size=3, activation='relu'))
                model_crs.add(MaxPooling1D(pool_size=3))
                BatchNormalization()
                model_crs.add(Conv1D(filters=256, kernel_size=3, activation='relu'))
                model_crs.add(MaxPooling1D(pool_size=3))
                BatchNormalization()
                model_crs.add(Conv1D(filters=512, kernel_size=3, activation='relu'))
                model_crs.add(MaxPooling1D(pool_size=3))
                '''model_crs.add(Conv1D(filters=512, kernel_size=3, activation='relu'))
                model_crs.add(MaxPooling1D(pool_size=3))
                model_crs.add(Conv1D(filters=1024, kernel_size=3, activation='relu'))
                model_crs.add(MaxPooling1D(pool_size=3))
                '''
                model_crs.add(Flatten())
                #model_crs.add(Dropout(0.5))
                model_crs.add(Dense(2000, activation='relu'))
                model_crs.add(Dense(n_outputs, activation='softmax'))
                model_crs.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
                # fit network   #CategoricalCrossentropy #sparse_categorical_crossentropy #SparseCategoricalCrossentropy
                earlystopping = callbacks.EarlyStopping(monitor ="val_accuracy", mode ="max", patience = 20, restore_best_weights = True)
                model_crs.fit(X_train, y_train_cat, epochs=epochs, batch_size=batch_size, verbose=verbose, validation_data = (X_valid, y_valid_cat), callbacks =[earlystopping])
                # evaluate model_crs
                _, accuracy = model_crs.evaluate(X_valid, y_valid_cat, batch_size=batch_size, verbose=verbose)
                predict_x=model_crs.predict(X_test)              # Function 1
                y_pred_crs=maxindx(predict_x)                    # function from augment.py to remove 0 index predictions
                rprt0=classification_report(y_test, y_pred_crs,output_dict=True)
                rprt_row=np.array([])
                rprt_row=np.append(rprt_row,rprt0['accuracy'])
                rprt_row=np.append(rprt_row,rprt0['macro avg']['f1-score'])
                rprt_row=np.append(rprt_row,rprt0['weighted avg']['f1-score'])
                rprt=np.append(rprt,[rprt_row],axis=0)
            #print('rprt=',rprt)
            maxm=rprt[np.argmax(rprt[:,1])]
            #print('maxm=',maxm)
            avrg=np.mean(rprt,axis=0)
            #print('avrg=',avrg)
            rslts=np.append(rslts,['X '+str(X)+'  alpha '+str(alpha) +'  Beta '+str(beta)+'  average= '+str(np.int16(100*avrg))+'  max= '+str(np.int16(100*maxm))+'  var= '+str(np.int16(np.var(100*rprt,axis=0)))],axis=0)
            print('rslts=',rslts[-1])


 max = 57
[[6 5 5 6 5]
 [1 1 1 1 1]
 [5 4 4 5 4]
 [5 4 4 5 4]
 [3 2 2 3 2]
 [3 3 3 3 3]
 [4 4 4 4 4]
 [2 2 2 2 2]
 [3 2 2 3 2]
 [0 0 0 0 0]
 [4 4 4 4 4]
 [4 4 4 4 4]
 [3 2 2 3 2]
 [4 4 4 4 4]
 [4 4 4 4 4]
 [4 4 4 4 4]
 [3 2 2 3 2]
 [2 2 2 2 2]
 [4 3 3 4 3]
 [2 1 1 2 1]
 [3 3 3 3 3]
 [4 3 3 4 3]
 [6 5 5 6 5]
 [4 4 4 4 4]
 [3 3 3 3 3]
 [4 4 4 4 4]
 [3 3 3 3 3]
 [4 4 4 4 4]
 [4 3 3 4 3]
 [9 8 8 9 8]
 [3 2 2 3 2]
 [5 4 4 5 4]
 [5 4 4 5 4]
 [1 1 1 1 1]
 [3 3 3 3 3]
 [5 5 5 5 5]
 [4 4 4 4 4]
 [3 3 3 3 3]
 [4 3 3 4 3]
 [2 2 2 2 2]
 [5 5 5 5 5]
 [2 2 2 2 2]]
62/62 [==============================] - 0s 3ms/step
rslts= X 1  alpha 1  Beta 1  average= [93 92 93]  max= [93 93 93]  var= [0 0 0]
[[ 6  4  4  6  4]
 [ 1  1  1  1  1]
 [ 4  3  3  4  3]
 [ 4  3  3  4  3]
 [ 2  2  2  2  2]
 [ 3  2  2  3  2]
 [ 5  3  3  5  3]
 [ 1  1  1  1  1]
 [ 2  2  2  2  2]
 [ 0  0  0  0  0]
 [ 5  3  3  5  3]
 [ 4  3  3  4  3]
 [ 2  2  2  2  2]
 [ 4  3  3  4  3]
 [ 4  3  3  4  3]
 [ 4  3  3  4  3]
 [ 2  1  1  2  1]
 [ 

62/62 [==============================] - 0s 3ms/step
rslts= X 1  alpha 3  Beta 6  average= [92 92 92]  max= [93 93 93]  var= [0 0 0]
[[21 18 18 21 18]
 [13 11 11 13 11]
 [20 17 17 20 17]
 [19 16 16 19 16]
 [16 14 14 16 14]
 [16 14 14 16 14]
 [17 14 14 17 14]
 [15 13 13 15 13]
 [16 13 13 16 13]
 [12 10 10 12 10]
 [17 14 14 17 14]
 [17 15 15 17 15]
 [15 13 13 15 13]
 [18 15 15 18 15]
 [17 15 15 17 15]
 [17 15 15 17 15]
 [17 14 14 17 14]
 [15 13 13 15 13]
 [16 14 14 16 14]
 [14 12 12 14 12]
 [16 14 14 16 14]
 [16 14 14 16 14]
 [20 17 17 20 17]
 [17 15 15 17 15]
 [16 13 13 16 13]
 [16 14 14 16 14]
 [16 14 14 16 14]
 [17 15 15 17 15]
 [18 15 15 18 15]
 [27 24 24 27 24]
 [15 13 13 15 13]
 [18 16 16 18 16]
 [17 15 15 17 15]
 [13 11 11 13 11]
 [16 13 13 16 13]
 [19 17 17 19 17]
 [16 14 14 16 14]
 [16 13 13 16 13]
 [17 15 15 17 15]
 [15 13 13 15 13]
 [19 17 17 19 17]
 [15 13 13 15 13]]
62/62 [==============================] - 0s 3ms/step
rslts= X 2  alpha 1  Beta 1  average= [93 92 93]  max= [9

62/62 [==============================] - 0s 3ms/step
rslts= X 2  alpha 3  Beta 6  average= [93 92 93]  max= [93 93 94]  var= [0 0 0]
[[35 30 30 35 30]
 [25 22 22 25 22]
 [35 30 30 35 30]
 [33 28 28 33 28]
 [29 25 25 29 25]
 [29 25 25 29 25]
 [29 25 25 29 25]
 [28 24 24 28 24]
 [29 24 24 29 24]
 [24 21 21 24 21]
 [29 25 25 29 25]
 [30 26 26 30 26]
 [28 24 24 28 24]
 [31 27 27 31 27]
 [30 26 26 30 26]
 [30 26 26 30 26]
 [31 26 26 31 26]
 [27 23 23 27 23]
 [28 24 24 28 24]
 [26 22 22 26 22]
 [28 24 24 28 24]
 [29 24 24 29 24]
 [34 29 29 34 29]
 [31 26 26 31 26]
 [28 24 24 28 24]
 [29 25 25 29 25]
 [30 25 25 30 25]
 [30 26 26 30 26]
 [32 27 27 32 27]
 [46 39 39 46 39]
 [27 23 23 27 23]
 [32 27 27 32 27]
 [30 25 25 30 25]
 [25 22 22 25 22]
 [28 24 24 28 24]
 [33 29 29 33 29]
 [29 25 25 29 25]
 [28 24 24 28 24]
 [30 26 26 30 26]
 [27 23 23 27 23]
 [33 28 28 33 28]
 [27 23 23 27 23]]
62/62 [==============================] - 0s 3ms/step
rslts= X 3  alpha 1  Beta 1  average= [93 93 93]  max= [9

62/62 [==============================] - 0s 3ms/step
rslts= X 3  alpha 3  Beta 6  average= [93 92 93]  max= [93 93 93]  var= [0 0 0]
[[49 42 42 49 42]
 [38 32 32 38 32]
 [49 42 42 49 42]
 [47 40 40 47 40]
 [43 37 37 43 37]
 [41 36 36 41 36]
 [41 35 35 41 35]
 [41 35 35 41 35]
 [41 35 35 41 35]
 [37 32 32 37 32]
 [41 35 35 41 35]
 [43 37 37 43 37]
 [40 35 35 40 35]
 [44 38 38 44 38]
 [43 37 37 43 37]
 [43 37 37 43 37]
 [45 39 39 45 39]
 [39 34 34 39 34]
 [40 34 34 40 34]
 [38 33 33 38 33]
 [41 35 35 41 35]
 [41 35 35 41 35]
 [48 41 41 48 41]
 [44 38 38 44 38]
 [40 34 34 40 34]
 [41 35 35 41 35]
 [43 37 37 43 37]
 [43 37 37 43 37]
 [46 39 39 46 39]
 [64 55 55 64 55]
 [40 34 34 40 34]
 [46 39 39 46 39]
 [42 36 36 42 36]
 [38 32 32 38 32]
 [41 35 35 41 35]
 [47 41 41 47 41]
 [41 35 35 41 35]
 [40 34 34 40 34]
 [43 37 37 43 37]
 [39 34 34 39 34]
 [47 40 40 47 40]
 [39 34 34 39 34]]
62/62 [==============================] - 0s 3ms/step
rslts= X 4  alpha 1  Beta 1  average= [93 93 93]  max= [9

62/62 [==============================] - 0s 3ms/step
rslts= X 4  alpha 3  Beta 2  average= [94 93 94]  max= [94 94 94]  var= [0 0 0]
[[ 59  23  23  59  23]
 [ 54  21  21  54  21]
 [ 57  23  23  57  23]
 [ 58  23  23  58  23]
 [ 55  22  22  55  22]
 [ 56  22  22  56  22]
 [ 59  23  23  59  23]
 [ 55  22  22  55  22]
 [ 56  22  22  56  22]
 [ 53  21  21  53  21]
 [ 59  23  23  59  23]
 [ 58  23  23  58  23]
 [ 56  22  22  56  22]
 [ 57  23  23  57  23]
 [ 58  23  23  58  23]
 [ 58  23  23  58  23]
 [ 54  21  21  54  21]
 [ 56  22  22  56  22]
 [ 58  23  23  58  23]
 [ 55  22  22  55  22]
 [ 57  23  23  57  23]
 [ 58  23  23  58  23]
 [ 59  24  24  59  24]
 [ 57  23  23  57  23]
 [ 57  23  23  57  23]
 [ 59  23  23  59  23]
 [ 55  22  22  55  22]
 [ 58  23  23  58  23]
 [ 56  22  22  56  22]
 [159  63  63 159  63]
 [ 57  23  23  57  23]
 [ 57  23  23  57  23]
 [ 59  23  23  59  23]
 [ 54  21  21  54  21]
 [ 56  22  22  56  22]
 [ 58  23  23  58  23]
 [ 59  23  23  59  23]
 [ 57  23  23  5

KeyboardInterrupt: 

In [10]:
print(rslts)

['X 1  alpha 1  Beta 1  average= [93 92 93]  max= [93 93 93]  var= [0 0 0]'
 'X 1  alpha 1  Beta 2  average= [92 92 92]  max= [93 93 93]  var= [0 0 0]'
 'X 1  alpha 1  Beta 6  average= [92 92 92]  max= [92 92 92]  var= [0 0 0]'
 'X 1  alpha 2  Beta 1  average= [93 92 93]  max= [93 92 93]  var= [0 0 0]'
 'X 1  alpha 2  Beta 2  average= [92 92 92]  max= [92 92 92]  var= [0 0 0]'
 'X 1  alpha 2  Beta 6  average= [92 92 92]  max= [92 92 92]  var= [0 0 0]'
 'X 1  alpha 3  Beta 1  average= [93 92 93]  max= [93 92 93]  var= [0 0 0]'
 'X 1  alpha 3  Beta 2  average= [92 92 92]  max= [93 92 93]  var= [0 0 0]'
 'X 1  alpha 3  Beta 6  average= [92 92 92]  max= [93 93 93]  var= [0 0 0]'
 'X 2  alpha 1  Beta 1  average= [93 92 93]  max= [93 93 93]  var= [0 0 0]'
 'X 2  alpha 1  Beta 2  average= [93 93 93]  max= [94 93 94]  var= [0 0 0]'
 'X 2  alpha 1  Beta 6  average= [92 92 92]  max= [93 93 93]  var= [0 0 0]'
 'X 2  alpha 2  Beta 1  average= [93 93 93]  max= [93 93 93]  var= [0 0 0]'
 'X 2  alpha